In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_algorithms.optimizers import COBYLA
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from sklearn.metrics import classification_report

# 1. Create binary data
n_samples = 100
n_features = 3
data = np.random.randint(2, size=(n_samples, n_features))
labels = np.random.randint(2, size=n_samples)
optimizer = COBYLA(maxiter=100)


# --- Prompt for IBM credentials and backend ---
api_key = input('Enter your IBM Quantum API key: ')
instance_name= input('Enter your Instance name: ')
backend_name = input('Enter backend name (e.g., ibm_brisbane): ')

# --- Try to connect to IBM Quantum backend ---
# --- Authenticate & get backend ---
try:
    service = QiskitRuntimeService(channel="ibm_cloud", token=api_key, instance=instance_name)
    backend = service.backend(backend_name)
    sampler = Sampler(backend)
    print(f"✅ Connected to {backend_name}")
except Exception as e:
    print(f"Error accessing IBM backend: {e}")
    print("Falling back to local AerSimulator.")
    backend = AerSimulator()
# Upload
uploaded = files.upload()

if uploaded:
    filename = next(iter(uploaded))
    df = pd.read_csv(filename, sep=None, engine='python')

    print("📄 File columns:", df.columns)

    # Try to remove unwanted index column
    if 'Unnamed: 0' in df.columns and df.shape[1] == 2:
        df = df.drop(columns='Unnamed: 0')

    if df.shape[1] >= 2:
        data = df.iloc[:, :-1].values
        labels = df.iloc[:, -1].values
        print("✅ Clean data loaded.")
    else:
        print("⚠️ Not enough columns, using random data.")
        data = np.random.randint(2, size=(n_samples, n_features))
        labels = np.random.randint(2, size=n_samples)
else:
    print("⚠️ No file uploaded. Using random binary data.")
    data = np.random.randint(2, size=(n_samples, n_features))
    labels = np.random.randint(2, size=n_samples)

print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")


# Split into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.3, random_state=42
)


print(f"Number of training samples: {len(train_data)}")
print(f"Number of testing samples: {len(test_data)}")
print(f"Data shape: {train_data.shape}") # Should be (n_samples, 2) for 2 features


# 4. Create quantum model
feature_map = ZZFeatureMap(n_features)
ansatz = TwoLocal(n_features, ['ry', 'rz'], 'cz', reps=3)

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    loss='cross_entropy',
    optimizer=optimizer
)

# 5. Train and evaluate
vqc.fit(train_data, train_labels)

# 2. Make predictions
train_preds = vqc.predict(train_data)
test_preds = vqc.predict(test_data)

# 3. Calculate accuracy
train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

# 4. Print results
print("Train accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)

# (Optional) Check predicted vs. true labels
print("Test predictions:", test_preds)
print("Test true labels:", test_labels)

print(classification_report(test_labels, test_preds))

# Plot
plt.bar(['Train Accuracy', 'Test Accuracy'], [train_accuracy, test_accuracy], color=['blue', 'green'])
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title("VQC Classification Accuracy")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()